In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [4]:
df['Sex'].value_counts()

Sex
I    33093
M    31027
F    26495
Name: count, dtype: int64

In [5]:
y = df['Rings']
df.drop(columns=['Rings', 'id'], inplace=True)

In [6]:
final_test = pd.read_csv('test.csv')
id = final_test['id']
final_test.drop(columns=['id'], inplace = True)
combined_df = pd.concat([df, final_test], axis=0)
combined_encoded = pd.get_dummies(combined_df, columns=['Sex'], prefix = 'encoded')

df_encoded = combined_encoded.iloc[:len(df), :]
final = combined_encoded.iloc[len(df):, :]




In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_encoded, y, train_size=0.8, random_state=42)

In [8]:
y_train.head()
X_train.head()

,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,encoded_F,encoded_I,encoded_M
43718,0.650,0.505,0.175,1.3850,0.7105,0.3020,0.360,False,False,True
45247,0.670,0.520,0.195,1.5405,0.6370,0.3455,0.385,True,False,False
71393,0.605,0.495,0.150,1.1345,0.5585,0.2470,0.285,False,False,True
51688,0.680,0.560,0.200,1.5340,0.6900,0.3060,0.440,False,False,True
40681,0.295,0.200,0.075,0.1075,0.0350,0.0190,0.038,False,True,False


In [9]:
from xgboost import XGBRegressor

xgb_reg = XGBRegressor(
    objective='reg:squarederror',  # Specify the objective as regression
    n_estimators=100,              # Number of boosting rounds (trees)
    learning_rate=0.1,             # Learning rate
    max_depth=5,                   # Maximum tree depth
    min_child_weight=1,            # Minimum sum of instance weight needed in a child
    gamma=0,                       # Minimum loss reduction required to make a further partition on a leaf node
    subsample=0.8,                 # Subsample ratio of the training instances
    colsample_bytree=0.8,          # Subsample ratio of columns when constructing each tree
    reg_alpha=0,                   # L1 regularization term on weights
    reg_lambda=1,                  # L2 regularization term on weights
    random_state=42                # Random seed for reproducibility
)

xgb_reg.fit(df_encoded, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [10]:
xgb_reg.score(X_test, y_test)

0.6733164577780699

In [11]:

prediction = xgb_reg.predict(final)
submission = pd.DataFrame(columns=['id', 'Rings'])
# id is just a list of ids
submission['id'] = id
submission['Rings'] = prediction

submission.to_csv('submission.csv', index=False)